## Initial Import

In [2]:
import pandas as pd
import numpy  as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## Data Cleaning

In [3]:
app = pd.read_csv('Base.csv')

In [4]:
app['fraud_bool'].value_counts(normalize=True)*100

0    98.8971
1     1.1029
Name: fraud_bool, dtype: float64

**It is observed that we have an extremely imbalanced dataset. Fraud transactions accounts for 1.1% of total rows.**

### Missing Values

1M record of bank account applications. No missing values in raw dataset.

In [5]:
app.include

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 32 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   fraud_bool                        1000000 non-null  int64  
 1   income                            1000000 non-null  float64
 2   name_email_similarity             1000000 non-null  float64
 3   prev_address_months_count         1000000 non-null  int64  
 4   current_address_months_count      1000000 non-null  int64  
 5   customer_age                      1000000 non-null  int64  
 6   days_since_request                1000000 non-null  float64
 7   intended_balcon_amount            1000000 non-null  float64
 8   payment_type                      1000000 non-null  object 
 9   zip_count_4w                      1000000 non-null  int64  
 10  velocity_6h                       1000000 non-null  float64
 11  velocity_24h                      1000

In [4]:
app.isnull().sum()

fraud_bool                          0
income                              0
name_email_similarity               0
prev_address_months_count           0
current_address_months_count        0
customer_age                        0
days_since_request                  0
intended_balcon_amount              0
payment_type                        0
zip_count_4w                        0
velocity_6h                         0
velocity_24h                        0
velocity_4w                         0
bank_branch_count_8w                0
date_of_birth_distinct_emails_4w    0
employment_status                   0
credit_risk_score                   0
email_is_free                       0
housing_status                      0
phone_home_valid                    0
phone_mobile_valid                  0
bank_months_count                   0
has_other_cards                     0
proposed_credit_limit               0
foreign_request                     0
source                              0
session_leng

In [5]:
app['month'].unique()

array([7, 3, 2, 1, 6, 0, 5, 4])

Since we don't have full year data and we shall predict whether an application is fraudulent base on its organic attributes, therefore, we will drop this column. 

In [6]:
app.drop(columns=['month'], inplace=True)

### Encode Categorical Columns

**Payment Type**

In [7]:
app['payment_type'].value_counts()

AB    370554
AA    258249
AC    252071
AD    118837
AE       289
Name: payment_type, dtype: int64

In [8]:
app = app.join(pd.get_dummies(app['payment_type']))

In [9]:
app.rename(columns={'payment_type':'pay_type', 'AA':'pay_type_AA', 'AB':'pay_type_AB', 'AC':'pay_type_AC', 
                    'AD':'pay_type_AD', 'AE':'pay_type_AE'}, inplace=True)

**Employment Status**

In [10]:
app['employment_status'].value_counts()

CA    730252
CB    138288
CF     44034
CC     37758
CD     26522
CE     22693
CG       453
Name: employment_status, dtype: int64

In [11]:
app = app.join(pd.get_dummies(app['employment_status']))

In [12]:
app.rename(columns={'employment_status':'emp_stat', 'CA':'emp_stat_CA', 'CB':'emp_stat_CB', 'CC':'emp_stat_CC',
                    'CD':'emp_stat_CD', 'CE':'emp_stat_CE', 'CF':'emp_stat_CF', 'CG':'emp_stat_CG'}, 
           inplace=True)

**Housing Status**

In [13]:
app['housing_status'].value_counts()

BC    372143
BB    260965
BA    169675
BE    169135
BD     26161
BF      1669
BG       252
Name: housing_status, dtype: int64

Fraud rate for BE, BF, BG are very similar, given small class size for BF, BG, BF and BG will be merged to class BE. 

In [14]:
def merge_class(i):
    if i == 'BF' or i == 'BG':
        return 'BE'
    else:
        return i

In [15]:
app['housing_status'] = app['housing_status'].apply(lambda x: merge_class(x))

In [16]:
app['housing_status'].unique()

array(['BA', 'BB', 'BC', 'BD', 'BE'], dtype=object)

In [17]:
app = app.join(pd.get_dummies(app['housing_status']))

In [18]:
app.rename(columns={'housing_status':'hous_stat', 'BA':'hous_stat_BA', 'BB':'hous_stat_BB', 'BC':'hous_stat_BC', 
                    'BD':'hous_stat_BD', 'BE':'hous_stat_BE'}, 
           inplace=True)

**Source**

In [19]:
app['source'].value_counts()

INTERNET    992952
TELEAPP       7048
Name: source, dtype: int64

**Notice that majority of applications come from 'INTERNET' source, therefore, dropping this column as it won't contribute much.**

In [20]:
app.drop(columns=['source'], inplace=True)

**Device OS**

In [21]:
app['device_os'].value_counts()

other        342728
linux        332712
windows      263506
macintosh     53826
x11            7228
Name: device_os, dtype: int64

In [22]:
app = app.join(pd.get_dummies(app['device_os']))

In [23]:
app.rename(columns={'other':'os_oth', 'linux':'os_lin', 'windows':'os_win', 'macintosh':'os_mac', 
                    'x11':'os_x11'}, 
           inplace=True)

### Continuous Variables

**Removing 'device_fraud_count' column since it only contains 1 value.**

In [24]:
app['device_fraud_count'].unique()

array([0])

In [25]:
app.drop(columns=['device_fraud_count'], inplace=True)

**Majority of applications are not originated from overseas. Remove this column.**

In [26]:
app['foreign_request'].value_counts()

0    974758
1     25242
Name: foreign_request, dtype: int64

In [27]:
app.drop(columns=['foreign_request'], inplace=True)

**Replace unreasonable values: average number of applications made in last 6 hours should not be negative. However, following the other column's formatting, negative application numbers will all be set to -1 which denotes no applications.**

In [28]:
app['velocity_6h'] = app['velocity_6h'].apply(lambda x: -1 if x<0 else x)